# Business Understanding
This dataset contains data for all NBA games from the 2004-2005 season until March of the 2021-2022 season.
From the description of the dataset, the data was collected from the NBA stats website.

This dataset can be useful for various parties, including NBA teams, coaches, players, sports bettors, or  anyone looking to better understand basketball statistics. According to www.investopedia.com, of the 30 NBA teams "a team is worth on average $2.12 billion", and "total revenue across the organization reached 8.76 billion in the 2018-2019 season."

In [4]:
import pandas as pd
import numpy as np

print('Pandas:', pd.__version__)
print('Numpy:',np.__version__)
df = pd.read_csv('NBA_datasets/games.csv') # read in the csv file
df.head(n = 100)

Pandas: 1.4.2
Numpy: 1.21.5


,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-03-12,22101005,Final,1610612748,1610612750,2021,1610612748,104.0,0.398,0.760,...,23.0,53.0,1610612750,113.0,0.422,0.875,0.357,21.0,46.0,0
1,2022-03-12,22101006,Final,1610612741,1610612739,2021,1610612741,101.0,0.443,0.933,...,20.0,46.0,1610612739,91.0,0.419,0.824,0.208,19.0,40.0,1
2,2022-03-12,22101007,Final,1610612759,1610612754,2021,1610612759,108.0,0.412,0.813,...,28.0,52.0,1610612754,119.0,0.489,1.000,0.389,23.0,47.0,0
3,2022-03-12,22101008,Final,1610612744,1610612749,2021,1610612744,122.0,0.484,0.933,...,33.0,55.0,1610612749,109.0,0.413,0.696,0.386,27.0,39.0,1
4,2022-03-12,22101009,Final,1610612743,1610612761,2021,1610612743,115.0,0.551,0.750,...,32.0,39.0,1610612761,127.0,0.471,0.760,0.387,28.0,50.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2022-02-28,22100925,Final,1610612760,1610612758,2021,1610612760,110.0,0.465,0.704,...,28.0,38.0,1610612758,131.0,0.563,0.767,0.323,30.0,46.0,0
96,2022-02-27,22100912,Final,1610612752,1610612755,2021,1610612752,109.0,0.447,0.629,...,21.0,47.0,1610612755,125.0,0.487,0.886,0.303,28.0,43.0,0
97,2022-02-27,22100913,Final,1610612756,1610612762,2021,1610612756,114.0,0.512,0.813,...,24.0,36.0,1610612762,118.0,0.468,0.650,0.425,23.0,41.0,0
98,2022-02-27,22100914,Final,1610612754,1610612738,2021,1610612754,128.0,0.522,0.850,...,25.0,49.0,1610612738,107.0,0.429,0.850,0.279,25.0,40.0,1
